In [1]:
import pandas as pd
import numpy as np
import pymysql 
import os
import glob
from fuzzywuzzy import process
import json
import datetime
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import xgboost as xgb
from tpot import TPOTRegressor
%matplotlib inline
pd.set_option('display.max_columns', 100)

/home/valesco/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/valesco/anaconda3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
### Load External Data Files
os.chdir('/home/valesco/Datasets/PGA_Data/pga_master_files/')

### Tournament Id's
tournament_ids = pd.read_csv('tournament_ids.csv', sep = ';')
tourny_ls = tournament_ids['tournament_name'].values

### Player Results
player_results = pd.read_csv('/home/valesco/Datasets/PGA_Data/pga_master_files/player_ranks.csv', sep = ';')

### Player Info
player_info = pd.read_csv('/home/valesco/Datasets/PGA_Data/pga_master_files/player_info.csv', sep = ',')

### Player Id's
player_ids = pd.read_csv('/home/valesco/Datasets/PGA_Data/pga_master_files/player_ids.csv', sep = ';')

### OWGR Ranks
owgr_ranks = pd.read_csv('/home/valesco/Datasets/PGA_Data/pga_master_files/owgr_final.csv', sep = ';')
owgr_ranks['player_id'] = owgr_ranks['player_id'].apply(lambda x: int(x))

In [3]:
def replace_buyin(x):
    if type(x) == float:
        return x
    else:
        return float(x.replace(',',''))
    
def add_player_id(df):
    player_names = player_ids['player_name'].unique()
    df_names = df['player_name'].unique()
    
    pid_dict = {}
    for player in df_names:
        pid_dict[player] = fuzz_match(player)
        
    df['player_id'] = df['player_name'].apply(lambda x: pid_dict[x])
    
    return df
    

def verify_input_df(df):
    temp_cols = df.columns
    for col in ['player_name', 'salary', 'tournament_name', 'tournament_id', 'buyin', 'player_id']:
        if col in temp_cols:
            assert True
    
    df['buyin'] = df['buyin'].apply(lambda x: replace_buyin(x))
    
    assert df['buyin'] == float
    assert df['salary'] == int or df['salary'] == float
    
    return df
    
    
def fuzz_match(x):
    matched = process.extractOne(x, player_names)
    if matched[1] > 89:
        return player_ids['player_id'].loc[player_ids['player_name'] == matched[0]].values[0]
    else:
        return np.nan
    

def preprocess_df(df):
    df = verify_input_df(df)
    df = add_player_id(df)
    
    return df

def prev_appear(df):
    player_id = df['player_id']
    tournament_id = df['tournament_id']
    date_cutoff = pd.to_datetime(df['date'])
    temp_df = player_results.loc[(player_results['player_id'] == player_id) & 
                                 (player_results['tournament_id'] == tournament_id) &
                                 (pd.to_datetime(player_results['tournament_start_date']) < date_cutoff)]

    num_appear = temp_df.shape[0]
    
    tourny_win = 0
    tourny_cuts_made = 0
    tourny_top_10 = 0
    
    for i in range(10):
        try:
            globals()['appear_' + str(i)] = temp_df['final_rank'].iloc[i]
        except:
            globals()['appear_' + str(i)] = -999
        
        try:
            if temp_df['final_rank'].iloc[i] == 1:
                tourny_win += 1

            if temp_df['final_rank'].iloc[i] > 0 and temp_df['final_rank'].iloc[i] < 21:
                tourny_top_10 += 1

            if temp_df['num_rounds_played'].iloc[i] == 4:
                tourny_cuts_made += 1
        except:
            pass
            
    try:        
        tourny_cut_percent = tourny_cuts_made / num_appear
        tourny_top_20_percent = tourny_top_10 / num_appear
    except:
        tourny_cut_percent = -999
        tourny_top_20_percent = -999
        
    
    return(num_appear, tourny_win, tourny_cut_percent, tourny_top_20_percent, appear_0, appear_1, 
          appear_2, appear_3, appear_4, appear_5, appear_6, appear_7, appear_8, appear_9)

def recent_results(df):
    try:
        player_id = df['player_id']
        date_cutoff = pd.to_datetime(df['date'])
        temp_df = player_results.loc[(player_results['player_id'] == player_id) & 
                                     (pd.to_datetime(player_results['tournament_start_date']) < date_cutoff)]

        temp_df.sort_values(by = 'tournament_start_date', ascending = False, inplace = True)
        
        recent_wins = 0
        
        for i in range(52):
            if i < 10: 
                try:
                    globals()['results_' + str(i)] = temp_df['final_rank'].iloc[i]
                except:
                    globals()['results_' + str(i)] = -999
            try:
                if temp_df['final_rank'].iloc[i] == 1:
                    recent_wins += 1
            except:
                pass

        return(recent_wins, results_0, results_1, results_2, results_3, results_4, results_5, results_6, 
              results_7, results_8, results_9)
    except:
        return(-999, -999, -999, -999, -999, -999, -999, -999, -999, -999, -999)
    
def recent_ownership(df):
    try:
        player_id = df['player_id']
        cutoff_date = pd.to_datetime(df['date'])
        temp_df = combo_df.loc[(combo_df['player_id'] == player_id) & (pd.to_datetime(combo_df['date']) < cutoff_date)]
        temp_df['date'] = pd.to_datetime(temp_df['date'])
        temp_df.sort_values(by = 'date', ascending = False, inplace = True)
        
        for i in range(10):
            try:
                globals()['own_' + str(i)] = temp_df['ownership'].iloc[i]
            except:
                globals()['own_' + str(i)] = -999
                
        return (own_0, own_1, own_2, own_3, own_4, own_5, own_6, own_7, own_8, own_9)
    except:
        return (-999, -999, -999, -999, -999, -999, -999, -999, -999, -999)
    
def map_owgr(df):
    try:
        player_id = df['player_id']
        tournament_id = df['tournament_id']
        date_cutoff = pd.to_datetime(df['date'])
        temp_df = owgr_ranks.loc[(owgr_ranks['player_id'] == player_id) & 
                                (pd.to_datetime(owgr_ranks['date']) < date_cutoff)]

        temp_df.sort_values(by = 'date', ascending = False, inplace = True)

        curr_rank = temp_df['rank'].iloc[0].mean()
        avg_3_rank = temp_df['rank'].iloc[:3].mean()
        avg_10_rank = temp_df['rank'].iloc[:10].mean()
        avg_25_rank = temp_df['rank'].iloc[:25].mean()
        avg_50_rank = temp_df['rank'].iloc[:50].mean()

        return(curr_rank, avg_3_rank, avg_10_rank, avg_25_rank, avg_50_rank)
    except:
        return(-999, -999, -999, -999, -999)
    
def elapsed(x):
    if x is not np.nan:
        time_elasped = datetime.datetime.now() - pd.to_datetime(x)
        return time_elasped.days
    else:
        return -999

def tourny_dummy(x):
    if x == 'GPP':
        return 0
    else:
        return 1

def return_major(x):
    majors_ls = ['US Open', 'British Open', 'Masters', 'PGA Championship']
    
    if x in majors_ls:
        return 1
    else:
        return 0
    
    

In [ ]:
test_df = pd.read('blahblah')
process_ownership(test_df)